In [1]:
# Group number: 24
# Franka Buytenhuijs, s4356845
# Hugo Chateau-Laurent, s1023970
# Maria Tsfasman, s1021505

**SOW-MKI49: Neural Information Processing Systems**

**Assignment 5: Neural Decoding**

**Total points: 100**

**Author: Umut**

In this assignment, you will implement a neural decoder for reconstructing perceived stimuli from brain responses. We will be using the dataset that was previously used in a number of papers. These papers along with additional lecture notes on neural decoding can be found in Brightspace. You can refer them for more details on the dataset and/or the method.

The dataset contains fMRI data acquired from the early visual cortex of one subject as the subject was presented with 100 grayscale images of handwritten sixes and nines (50 sixes and 50 nines). The fMRI data has been realigned and slice time corrected. Furthermore, stimulus specific response amplitudes have been estimated with a general linear model.

Let's first familiarize ourselves with the dataset. It contains a number of variables:

* **X** -> This is a 100 x 784 matrix. The ith row contains the pixel values of the stimulus that was presented in the ith trial of the experiment. Note that the stimuli are 28 pixel x 28 pixel images, which were reshaped to 1 x 784 vectors.

* **Y** -> This is a 100 x 3092 matrix. The ith row contains the voxel values of the responses that were measured in the ith trial of the experiment.

* **X_prior** -> This is a 2000 x 784 matrix. Each row contains the pixel values of a different stimulus, which was not used in the experiment. Note that the stimuli are 28 pixel x 28 pixel images, which were reshaped to 1 x 784 vectors.

Note: In the remainder of this document, we will use **x** for referring to a 784 x 1 stimulus vector and **y** for referring to a 3092 x 1 response vector.

## Task 1 (10 points)

* Load the dataset.
* Visualize some of the stimuli.
* Normalize X and Y to have zero mean and unit variance. Tip: Recall that normalization means subtracting the mean of each pixel/voxel from itself and dividing it by its standard deviation. You can use zscore function.
* Split X and Y in two parts called X_training and X_test, and Y_training and Y_test. The training set should contain 80 stimulus-response pairs (40 pairs for sixes and 40 pairs for nines). The test set should contain 20 stimulus-response pairs (10 pairs for sixes and 10 pairs for nines).

In [2]:
gpu = False

if gpu:
    import cupy as np
else:
    import numpy as np
    
from scipy.io import loadmat
from scipy.stats import zscore

import scipy

from chainer import cuda, datasets, serializers, optimizers
from chainer.dataset import DatasetMixin, concat_examples
import chainer.iterators as iterators
from chainer.iterators import MultithreadIterator

import chainer
import chainer.functions as F
import chainer.links as L

import math
import matplotlib.pyplot as plt
import seaborn as sns


ImportError: No module named 'seaborn'

In [ ]:
def prepare(arr):
    # prepare array to plot it the right way
    return np.flipud(np.rot90(arr.reshape(28,-1),1))


data = scipy.io.loadmat('69dataset2.mat')

X = zscore(data['X'], axis = None) # normalize X
Y = zscore(data['Y'], axis=None) #normalize Y
X_prior = zscore(data['X_prior'], axis=None) #normalize X_prior


# print all keys
print('Keys in data:')
for key in data:
    print('  '+key)

plt.figure(figsize=(18,6))
tot = 10
for i in range(tot):   
    
    #Plot 6
    plt.subplot(2,tot,1+i)    
    plt.imshow(prepare(X[i]), cmap='gray')
    plt.xticks([])
    plt.yticks([])
    
    #Plot 9
    plt.subplot(2,tot,1+i+tot)  
    plt.imshow(prepare(X[i+50]), cmap='gray')
    plt.xticks([])
    plt.yticks([])

plt.show()


X_training = np.concatenate((X[:40], X[50:90]))
X_test = np.concatenate((X[40:50], X[90:]))

Y_training = np.concatenate((Y[:40], Y[50:90]))
Y_test = np.concatenate((Y[40:50], Y[90:]))



##### Our goal is to solve the problem of reconstructing **x** from **y**. One possible approach to solve this problem is to use a *discriminative* model. Discriminative models predict **x** as a function of **y**. That is:

**x** = f(**y**)

We will assume that f is a linear function. That is:

**x** = **B'** **y**

f can be seen as a very simple linear neural network comprising one layer of weights (i.e., **B**). We can estimate **B** in close form with ridge regression. That is:

**B** = inv(**Y**\_training' **Y**\_training + lambda **I**) **Y**\_training' **X**\_training

where lambda is the regularization coefficient, **I** is the *q* x *q* identity matrix, and *q* is the number of voxels. Note that we can safely ignore the intercept since we normalized our data to have zero mean and unit variance.

## Task 2  (15 points)

* Estimate **B** on the training set. Tip: Normally, you should use cross validation to estimate lambda. For simplicity, you can assume that lambda = 10 ^ -6.
* Reconstruct **x** from **y** in the test set.
* Visualize the reconstructions.

In [ ]:
#Define q and lambda
q = np.size(Y_training, 1)
print(Y_training.shape)
lamb = math.pow(10,-6)

#Estimate B on the training set
# use lstq
B_temp = np.linalg.inv(np.matmul(np.transpose(Y_training), Y_training) + lamb * np.identity(q)) 
B = np.matmul(np.matmul(B_temp, np.transpose(Y_training)), X_training)

#Reconstruct x from y in the test set.
X_reconstruct = np.matmul(Y_test, B)

print(X_reconstruct.shape)
#Visualize the reconstructions
plt.figure(figsize=(18,6))
tot = 10
for i in range(tot):   
    
    #Plot x_test
    plt.subplot(2,tot,1+i)    
    plt.imshow(prepare(X_test[i]), cmap='gray')
    
    #Plot x_reconstruct
    plt.subplot(2,tot,1+i+tot)  
    plt.imshow(prepare(X_reconstruct[i]), cmap='gray')
    
plt.show()


Another possible approach to solve the problem of reconstructing **x** from **y** is to use a *generative* model and invert it with Bayes' rule. We reformulate the problem as finding the most probable **x** that could have caused **y**. That is:

argmax_**x** P(**x** | **y**)

where P(**x** | **y**) is called the posterior (probability of the stimulus being **x** if the observation is **y**). In other words, we have to define the posterior, estimate its parameters and find the argument that maximizes it, which will be the reconstruction of **x** from **y**. While, this may seem daunting, it actually has a simple solution. The posterior assigns a probability to an event by combining our observations and beliefs about it, and can be decomposed with Bayes' rule as the product of how likely our observations are given the event (probability of observing **y** if the stimulus is **x**) and how likely the event is independent of our observations (probability of the stimulus being **x**). That is:

P(**x** | **y**) ~ P(**y** | **x**) * P(**x**)

where P(**y** | **x**) is called the likelihood and P(x) is called the prior.

We will assume that the likelihood and the prior are multivariate Gaussian distributions. A Gaussian is characterized by two parameters: a mean vector and a covariance matrix. In the case of the likelihood, the mean of the Gaussian is given by:

**mu**\_likelihood = **B'** **x**

As before, we can estimate **B** in close form with ridge regression:

**B** = inv(**X**\_training' **X**\_training + lambda **I**) **X**\_training' **Y**\_training

where lambda is the regularization coefficient, I is the *p* x *p* identity matrix, and *p* is the number of pixels. The covariance matrix of the likelihood is given by:

**Sigma**_likelihood = diag(E[||**y** - **B'** **x**|| ^ 2]). 

In the case of the prior, the mean of the Gaussian is given by:

**mu**\_prior = **0** (which is a vector of zeros)

The covariance matrix of the prior is given by:

**Sigma**\_prior = **X**\_prior' * **X**\_prior / (n - 1)

where n is the length of **X**\_prior.

## Task 3 (20 points)

* Estimate **B** on the training set. Tip: Normally, you should use cross-validation to estimate lambda and Sigma_likelihood. For simplicity, you can assume that lambda = 10 ^ -6 and Sigma_likelihood = 10 ^ -3 **I**.
* Estimate **Sigma**\_prior. Tip: Add 10 ^ -6 to the diagonal of Sigma_prior for regularization.
* Visualize **Sigma**\_prior. Can you explain what it shows?

In [ ]:
#Define lamda, sigma and q
p = np.size(X_training, 1)
print(X_training.shape)
print(p)
lamb = math.pow(10,-6)
sigma_like = math.pow(10, -3) * np.identity(p)
print(sigma_like.shape)


#Estimate B on the training set
#B = inv(X_training' X_training + lambda I) X_training' Y_training
B_temp = np.linalg.inv(np.matmul(np.transpose(X_training), X_training) + lamb * np.identity(p)) 
B = np.matmul(np.matmul(B_temp, np.transpose(X_training)), Y_training)
print(B.shape)

#Estimate sigma_prior
n = np.size(X_prior)
sigma_prior = np.matmul(np.transpose(X_prior), X_prior) / (n-1)
sigma_prior_reg = sigma_prior + lamb * np.identity(p)
print(sigma_prior_reg.shape)

plt.imshow(sigma_prior_reg)
plt.show()

#Explain what sigma_prior shows
#TODO

Having defined the likelihood and the prior as Gaussians, we can derive the posterior by multiplying them. It turns out that the product of two Gaussians is another Gaussian, whose mean vector is given by:

**mu**\_posterior = inv(inv(**Sigma**\_prior) + **B** inv(**Sigma**\_likelihood) **B**') **B** * inv(**Sigma**\_likelihood) **y**

We are almost done. Recall that the reconstruction of **x** from **y** is the argument that maximizes the posterior, which we derived to be a Gaussian. We will be completely done once we answer the following question: What is the argument that maximizes a Gaussian?

.  
.  
.  
.  
.  
.

The answer is its mean vector, which is the solution of our initial problem. That is:

argmax_**x** P(**x** | **y**) =  
**mu**\_posterior =  
inv(inv(**Sigma**\_prior) + **B** inv(**Sigma**\_likelihood) **B**') **B** * inv(**Sigma**\_likelihood) **y**

Now, we can plug any **y** in the above equation and reconstruct the most probable **x** that could have caused it.


## Task 4 (25 points)

- Reconstruct **x** from **y** in the test set.
- Visualize the reconstructions.
- Compare the reconstructions with the earlier reconstructions. Which one is better? Why? Can you think of ways to improve the results?

In [ ]:
#Reconstruct x from y in the test set.
#mu_posterior = inv(inv(Sigma_prior) + B inv(Sigma_likelihood) B') B inv(Sigma_likelihood) *y
#had some problems with matrix dimensions here. Could not exactly do what is described above

X_test_reconstructed_temp = np.linalg.inv(np.linalg.inv(sigma_prior_reg) + np.matmul(np.matmul(np.linalg.inv(sigma_like), B), np.transpose(B)))
X_test_reconstructed = np.matmul(np.transpose(np.matmul(np.matmul(X_test_reconstructed_temp, B).T, np.linalg.inv(sigma_like))), np.transpose(Y_test))
X_test_reconstructed_trans = np.transpose(X_test_reconstructed)

plt.figure(figsize=(18,6))
tot = 10
for i in range(tot):   
    
    #Plot x_test
    plt.subplot(2,tot,1+i)    
    plt.imshow(prepare(X_test[i]), cmap='gray')
    
    #Plot x_reconstruct
    plt.subplot(2,tot,1+i+tot)  
    plt.imshow(prepare(X_test_reconstructed_trans[i]), cmap='gray')
    
plt.show()

Now the real fight begins! You might have already realized that the second approach is very similar to the one in the face reconstruction paper that we have discussed in the last lecture. The main difference is how the stimuli are encoded. So far, we have been encoding the stimuli with an identity transformation:

**mu**\_likelihood = **B'** g(**x**)

where g(**x**) = **x**

In the paper, they are encoded as the features extracted from a DNN:

**mu**\_likelihood = **B'** DNN(**x**)

Similarly, we have been decoding the responses with MAP estimation of the stimuli:

g ^ -1(argmax_g(**x**) P(g(**x**) | **y**))

where g ^ -1(**y**) = **y**

In the paper, they are decoded with MAP estimation of the features followed by an inverse DNN:

DNN ^ -1(argmax_DNN(**x**) P(DNN(**x**) | **y**))

## Task 5 (30 points)

- Train a simple DNN for digit classification on the MNIST dataset.
- Select an indermediate layer of the DNN as your feature extractor.
- Train another simple DNN to invert the first DNN. That is, it should transform features extracted by the selected layer of the first DNN to stimuli.
- Repeat Task 3 but use the trained DNNs instead of identity transformation.

In [11]:
dropout = .2 # global dropout rate

class Decoder(chainer.ChainList):
    def __init__(self, n_hlay, n_z, n_h, n_x): # <= you might want to pass some architecture parameters (e.g., #i/o units, etc.) here
        
        # Parameters:
        #  n_hlay: number of hidden layers
        #  n_z: dimension of latent space
        #  n_h: dimension of hidden layers (if n_hlay>0)
        #  n_x: dimension of x_out (=dimension of x_in)
        
        links=()
        
        if(n_hlay>0):
            links += (L.Linear(n_z, n_h),)
            for _ in range(n_hlay-1):
                links += (L.Linear(n_h, n_h),)
            n_z = n_h
        
        links += (L.Linear(n_z, n_x),)
        
        print('Decoder:', len(links))
                
        super(Decoder, self).__init__(*links)

    def __call__(self, z):         
        for layer in self[:-2]:
            z = F.relu(layer(z))   
            z = F.dropout(z, dropout)
        z = self[-1](z)       
        
        return z

class Encoder(chainer.ChainList):
    def __init__(self, n_hlay, n_x, n_h, n_z):
        # Parameters:
        #  n_hlay: number of hidden layers
        #  n_x: dimension of x_out (=dimension of x_in)
        #  n_h: dimension of hidden layers (if n_hlay>0)
        #  n_z: dimension of latent space

        links=()
        
        if(n_hlay>0):
            links += (L.Linear(n_x, n_h),)
            for _ in range(n_hlay-1):
                links += (L.Linear(n_h, n_h),)
            n_x = n_h
        
        links += (L.Linear(n_x, n_z),)
        
        print('Encoder:', len(links))
                
        super(Encoder, self).__init__(*links)

    def __call__(self, x):  
        for layer in self[:-2]:
            x = F.relu(layer(x))
            x = F.dropout(x, dropout)
        
        x = self[-1](x)     
        
        return x
    
class Loss_encoder(object):
    def __call__(self, y, t):
        return softmax_cross_entropy(y,t)

    
class Loss_decoder(object):
    def __call__(self, y, t):
        return softmax_cross_entropy(y,t)
    
    
class Mnist(DatasetMixin):
    def __init__(self, start_proportion, end_proportion):
        dataset = datasets.get_mnist(False)[0 if chainer.config.train else 1]
        size = dataset.shape[0]
        self.dataset = dataset[int(size*start_proportion):int(size*end_proportion),:]

    def __len__(self):
        return len(self.dataset)

    def get_example(self, i):
        return self.dataset[i]
    
class Optimizer():
    def __init__(self, model, loss):
        self.model = model
        self.log = {'training':[], 'test':[]}   
        self.optimizer = optimizers.Adam()
        self.optimizer.setup(self.model)
        self.loss = loss 
        
    def test(self, x, t):
        with chainer.using_config('train', False):
            y = self.model(x)
            err = self.loss(y, t)   
            return float(err.data)
    
    def train(self, x, t):
        err = self.loss(self.model(x), t) # Forward propagation 
        self.model.cleargrads()
        err.backward() # backpropagation
        self.optimizer.update() 
        return float(err.data)
    
    @classmethod
    def load(cls, model, loss, directory):
        self = cls(model)
        self.log = np.load('{}/log.npy'.format(directory))
        serializers.load_npz('{}/weights.npz'.format(directory), self.model)
        serializers.load_npz('{}/optimizer.npz'.format(directory), self.optimizer)

        return self

    def save(self, directory):
        np.save('{}/log.npy'.format(directory), self.log)
        serializers.save_npz('{}/weights.npz'.format(directory), self.model)
        serializers.save_npz('{}/optimizer.npz'.format(directory), self.optimizer)

# model from scratch
encoder = Encoder(2,784,500,20) # (n_hlay, n_x, n_h, n_z)
decoder = Decoder(2,20,500,784) # (n_hlay, n_z, n_h, n_x)

if gpu:
    encoder.to_gpu()
    decoder.to_gpu()
    


load = False
if load:
    optimizer_encoder = Optimizer.load(encoder, Loss_encoder(), "./MNIST/Weights_0/15")
    optimizer_decoder = Optimizer.load(decoder, Loss_decoder(), "./MNIST/Weights_0/15")
else:
    optimizer_encoder = Optimizer(encoder, Loss_encoder())
    optimizer_decoder = Optimizer(decoder, Loss_decoder())
if gpu:
    optimizer.model.to_gpu() 

epochs = 100
batch_size = 100

training_set = Mnist(0.,.8)
validation_set = Mnist(.8,.9)
test_set = Mnist(.9,1.)

training_iterator = iterators.SerialIterator(training_set, batch_size, False, True)
validation_iterator = iterators.SerialIterator(validation_set , batch_size, False, False)

Encoder: 3
Decoder: 3


In [ ]:
# Train encoder
root = '.'
i=-1
path = '{}/Models'.format(root)
if not os.path.isdir(path):
    os.makedirs(path)
while i==-1 or os.path.isdir(path):
    i+=1
    path = '{}/Encoder/Models/Weights_{}'.format(root, i)
os.makedirs('{}/Encoder/Models/Weights_{}'.format(root, i))

optimizer.log['training'] = []
optimizer.log['test'] = []

for epoch in tqdm.tnrange(epochs):
    
    training_iterator.reset()
    test_iterator.reset()
    
    mean_loss = []
    for j, batch in enumerate(training_iterator):
        mean_loss.append(optimizer.train(np.array(batch)))
    optimizer.log['training'] += [numpy.mean(mean_loss)]

    mean_loss = []
    for j, batch in enumerate(test_iterator):
        mean_loss.append(optimizer.test(np.array(batch)))
    optimizer.log['test'] += [numpy.mean(mean_loss)]
    
    IPython.display.clear_output()
    
    plt.figure(figsize=(18,6))
    samples_toshow = 8
    for i in range(samples_toshow):
        plt.subplot(2,samples_toshow,1+i)
        optimizer.test(np.array(batch), show=i)
        plt.subplot(2,samples_toshow,1+samples_toshow+i)
        plt.imshow(batch[i].reshape(28,-1), cmap='gray')
    plt.title("Epoch "+str(epoch+1))
    plt.show()
    
    if False:
        # store weights and loss
        os.makedirs('{}/{}'.format(path, epoch))
        optimizer.save('{}/{}'.format(path, epoch))
    
    plt.plot(range(epoch+1), optimizer.log['test'], label='test')
    plt.plot(range(epoch+1), optimizer.log['training'], label='training')
    plt.xlabel('iteration')
    plt.ylabel('loss')
    plt.legend()    
    plt.show()

In [ ]:
# Train decoder

root = '.'
i=-1
path = '{}/Models'.format(root)
if not os.path.isdir(path):
    os.makedirs(path)
while i==-1 or os.path.isdir(path):
    i+=1
    path = '{}/Models/Weights_{}'.format(root, i)
os.makedirs('{}/Models/Weights_{}'.format(root, i))

optimizer.log['training'] = []
optimizer.log['test'] = []

for epoch in tqdm.tnrange(epochs):
    
    training_iterator.reset()
    test_iterator.reset()
    
    mean_loss = []
    for j, batch in enumerate(training_iterator):
        mean_loss.append(optimizer.train(np.array(batch)))
    optimizer.log['training'] += [numpy.mean(mean_loss)]

    mean_loss = []
    for j, batch in enumerate(test_iterator):
        mean_loss.append(optimizer.test(np.array(batch)))
    optimizer.log['test'] += [numpy.mean(mean_loss)]
    
    IPython.display.clear_output()
    
    plt.figure(figsize=(18,6))
    samples_toshow = 8
    for i in range(samples_toshow):
        plt.subplot(2,samples_toshow,1+i)
        optimizer.test(np.array(batch), show=i)
        plt.subplot(2,samples_toshow,1+samples_toshow+i)
        plt.imshow(batch[i].reshape(28,-1), cmap='gray')
    plt.title("Epoch "+str(epoch+1))
    plt.show()
    
    if False:
        # store weights and loss
        os.makedirs('{}/{}'.format(path, epoch))
        optimizer.save('{}/{}'.format(path, epoch))
    
    plt.plot(range(epoch+1), optimizer.log['test'], label='test')
    plt.plot(range(epoch+1), optimizer.log['training'], label='training')
    plt.xlabel('iteration')
    plt.ylabel('loss')
    plt.legend()    
    plt.show()